In [2]:
import torch
from torch import nn,optim
from torchvision import datasets,transforms
from torch.utils.data import dataloader
import matplotlib.pyplot as plt
from tqdm import tqdm






In [3]:
my_transform = transforms.Compose([transforms.ToTensor(),transforms.Lambda(lambda img: img.reshape(784))])

In [4]:
training_set = datasets.MNIST(root="data",train=True,transform=my_transform,download=True)
testing_set  = datasets.MNIST(root="data",train=False,transform=my_transform,download=True)


100%|██████████| 9912422/9912422 [00:00<00:00, 156256196.56it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 40513609.97it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 40535072.51it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17350208.35it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [5]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 16

In [6]:
class PyTeen(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1,6,5,padding=2),
      nn.ReLU(),
      nn.MaxPool2d(2,stride=2),

      nn.Conv2d(6,16,5,padding=0),
      nn.ReLU(),
      nn.MaxPool2d(2,stride=2),

      nn.Flatten(),
      nn.Linear(400,120),
      nn.Linear(120,84),
      nn.Linear(84,10)


    )
    self.loss = nn.CrossEntropyLoss()
    self.optimizer = optim.Adam(self.parameters())
   #self.to(torch.device(DEVICE)) #gpu

  def forward(self,input):
    return(self.layers(input))

  def predict(self,input):
    with torch.no_grad():
      pred = self.forward(input)
      return(torch.argmax(pred,axis=-1))

  def train(self,input,label):
    self.optimizer.zero_grad()
    pred = self.forward(input)
    loss = self.loss(pred,label)
    loss.backward()
    self.optimizer.step()
    return(loss)




In [7]:
training_loader = dataloader.DataLoader(training_set,batch_size=BATCH_SIZE,shuffle=True)
testing_loader = dataloader.DataLoader(testing_set,batch_size=BATCH_SIZE,shuffle=False)

In [8]:
network = PyTeen()
network.to(torch.device(DEVICE))


PyTeen(
  (layers): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=400, out_features=120, bias=True)
    (8): Linear(in_features=120, out_features=84, bias=True)
    (9): Linear(in_features=84, out_features=10, bias=True)
  )
  (loss): CrossEntropyLoss()
)

In [9]:
# training loop
# we need to make the tensors all .to(torch.device('cuda'))
EPOCHS = 10

for i in range(EPOCHS):
  total_loss = 0
  for input,label in tqdm(training_loader):
      input = input.to(torch.device(DEVICE))
      label = label.to(torch.device(DEVICE))

      loss = network.train(input,label)
      total_loss += loss
  print("EPOCH:",i+1,": ",total_loss)





  0%|          | 0/3750 [00:00<?, ?it/s]


RuntimeError: ignored

In [ ]:
# evaluation loop
# we need to make the tensors all .to(torch.device('cuda'))
num_corrects = 0
for input,label in tqdm(testing_loader):
  input = input.to(torch.device(DEVICE))
  label = label.to(torch.device(DEVICE))
  pred = network.predict(input)
  for i in range(len(pred)):
    if(pred[i] == label[i]):
      num_corrects += 1
print('\n',num_corrects/(len(testing_loader)*BATCH_SIZE))
